In [1]:
import bittensor as bt
import storage
from storage.validator.database import *
from storage.validator.bonding import *


In [2]:
import redis.asyncio as redis
database= r = redis.StrictRedis(db=8)

In [3]:
await current_validator_storage(r)

1063359477167

In [4]:
await get_network_capacity(r)

11529354684045197312

In [6]:
size_bytes = await get_redis_db_size(r)
size_bytes / (1024 * 1024) # MB

65.73680305480957

In [5]:
all_hashes = await get_all_hashes_in_database(r)
len(all_hashes)

41276

In [8]:
challenge_hashes = await get_all_challenge_hashes(r)
len(challenge_hashes)

41276

In [5]:
stats = await get_miner_statistics(r)
len(stats)

38

In [7]:
stats

{'5Cih6V8oc7ahYHAXwnSVu8ys8h3GM4A1kXJqfJFQCwD9XmFy': {'store_attempts': '128',
  'store_successes': '128',
  'challenge_successes': '73',
  'challenge_attempts': '78',
  'retrieve_successes': '7',
  'retrieve_attempts': '10',
  'total_successes': '2261',
  'tier': 'Gold',
  'storage_limit': '109951162777600'},
 '5CoEiuGiXZgeFSpccqo17aiovSRh9Tv2Vn6j65J93iEhB2hJ': {'store_attempts': '0',
  'store_successes': '0',
  'challenge_successes': '0',
  'challenge_attempts': '81',
  'retrieve_successes': '0',
  'retrieve_attempts': '0',
  'total_successes': '1998',
  'tier': 'Bronze',
  'storage_limit': '1099511627776'},
 '5Do7YsNrKEpejfTMsmaCxh9FPHrxgBi4jYh9VcbeifobjNAe': {'store_attempts': '0',
  'store_successes': '0',
  'challenge_successes': '0',
  'challenge_attempts': '0',
  'retrieve_successes': '0',
  'retrieve_attempts': '0',
  'total_successes': '0',
  'tier': 'Bronze',
  'storage_limit': '1099511627776'},
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon': {'store_attempts': '140',
 

In [6]:
for k,v in stats.items():
    print(v)

{'store_attempts': '128', 'store_successes': '128', 'challenge_successes': '73', 'challenge_attempts': '78', 'retrieve_successes': '7', 'retrieve_attempts': '10', 'total_successes': '2261', 'tier': 'Gold', 'storage_limit': '109951162777600'}
{'store_attempts': '0', 'store_successes': '0', 'challenge_successes': '0', 'challenge_attempts': '81', 'retrieve_successes': '0', 'retrieve_attempts': '0', 'total_successes': '1998', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '0', 'store_successes': '0', 'challenge_successes': '0', 'challenge_attempts': '0', 'retrieve_successes': '0', 'retrieve_attempts': '0', 'total_successes': '0', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '140', 'store_successes': '140', 'challenge_successes': '78', 'challenge_attempts': '78', 'retrieve_successes': '15', 'retrieve_attempts': '15', 'total_successes': '27449', 'tier': 'Super Saiyan', 'storage_limit': '1152921504606846976'}
{'store_attempts': '0', 'store_suc

In [41]:
# Get storage capacity of network based on miner stats
async def get_network_capacity(database):
    """
    Get the total storage capacity of the network in Terabytes.
    """
    cap = 0
    for k,v in (await get_miner_statistics(database)).items():
        cap += int(v.get('storage_limit', 0))

    return cap / (1024 ** 3) # GB

await get_network_capacity(r) #/ (1024 ** 2) # PB

10737548288.0

In [26]:
async def active_hotkeys(database):
    return [x.decode().split(":")[-1] async for x in r.scan_iter("hotkey:*")]

In [53]:
async def get_hash_keys(ss58_address, database):
    """
    Filter out the ttl: hashes from the hotkey hashes and return the list of keys.
    """
    return [
        key for key in await database.hkeys(f"hotkey:{ss58_address}") if not key.startswith(b"ttl:")
    ]

await get_hash_keys(hotkeys[0], r)

[b'...',
 ...]

In [27]:
hotkeys = await active_hotkeys(r)
hotkeys

['5Cih6V8oc7ahYHAXwnSVu8ys8h3GM4A1kXJqfJFQCwD9XmFy',
 '5FRdgnVriaM2SUBAv4aUyEGrGJqjFGMq96dp3myEJsrM745j',
 '5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2',
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon',
 '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz',
 '5CK49c4ihRkCe4wf7DpqzxA1WP5gpiu6QF1V2zxdVZMonHeC',
 '5Gpjcz44ViQJzjA14xAgMSZwTLeULeJdXhjKMV6JM3XrGkUw',
 '5CoEiuGiXZgeFSpccqo17aiovSRh9Tv2Vn6j65J93iEhB2hJ',
 '5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i',
 '5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX',
 '5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1',
 '5H1CaMTf6XvzSYLcLe4AhnXwttQ4z3ry6fy9XzS9wrxV5vFA',
 '5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK',
 '5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU',
 '5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev']

In [30]:
caps = await cache_hotkeys_capacity(hotkeys, r)
caps

/home/phil/miniconda3/envs/dash/lib/python3.11/asyncio/futures.py:172: RuntimeWarning: coroutine 'get_miner_statistics' was never awaited
  for callback, ctx in callbacks:


{'5Cih6V8oc7ahYHAXwnSVu8ys8h3GM4A1kXJqfJFQCwD9XmFy': (11370725711,
  109951162777600),
 '5FRdgnVriaM2SUBAv4aUyEGrGJqjFGMq96dp3myEJsrM745j': (24726815602,
  1099511627776),
 '5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2': (92020653634,
  1152921504606846976),
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon': (101956336549,
  1152921504606846976),
 '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH': (4469444834,
  1099511627776),
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz': (98340000712,
  1152921504606846976),
 '5CK49c4ihRkCe4wf7DpqzxA1WP5gpiu6QF1V2zxdVZMonHeC': (44594794, 1099511627776),
 '5Gpjcz44ViQJzjA14xAgMSZwTLeULeJdXhjKMV6JM3XrGkUw': (51800577088,
  1099511627776),
 '5CoEiuGiXZgeFSpccqo17aiovSRh9Tv2Vn6j65J93iEhB2hJ': (11919267688,
  1099511627776),
 '5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i': (100268508208,
  1152921504606846976),
 '5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX': (99155923764,
  1152921504606846976),
 '5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWs

In [54]:
async def current_validator_storage(database):
    """
    Get the total storage capacity of the network in Terabytes.
    """
    hotkeys = await active_hotkeys(database)

    current_storage = 0
    for k, (cur, _) in (await cache_hotkeys_capacity(hotkeys, database)).items():
        current_storage += cur

    return current_storage / (1024 ** 3) # GB


await current_validator_storage(r)

990.3306860169396

In [26]:
# Tier breakdown
from typing import Dict
async def tier_statistics(databse: aioredis.Redis) -> Dict[str, Dict[str, int]]:
    tier_counts = {
        "Super Saiyan": 0,
        "Diamond": 0,
        "Gold": 0,
        "Silver": 0,
        "Bronze": 0,
    } 
    tier_capacity = {
        "Super Saiyan": 0,
        "Diamond": 0,
        "Gold": 0,
        "Silver": 0,
        "Bronze": 0,
    }
    tier_usage = {
        "Super Saiyan": 0,
        "Diamond": 0,
        "Gold": 0,
        "Silver": 0,
        "Bronze": 0,
    }

    for k,v in (await get_miner_statistics(database)).items():
        tier = v.get('tier', None)
        if tier:
            hotkey = k.split(":")[-1]
            tier_counts[tier] += 1
            tier_capacity[tier] += int(v.get('storage_limit', 0))
            tier_usage[tier] += await total_hotkey_storage(hotkey, database, False)

    tier_percent_usage = {k: 100 * (v / tier_capacity[k]) if tier_capacity[k] > 0 else 0 for k,v in tier_usage.items()}

    return {
        "counts": tier_counts, 
        "capacity": tier_capacity, 
        "usage": tier_usage,
        "percent_usage": tier_percent_usage
    }

In [27]:
await tier_statistics(r)

{'counts': {'Super Saiyan': 10,
  'Diamond': 0,
  'Gold': 1,
  'Silver': 0,
  'Bronze': 27},
 'capacity': {'Super Saiyan': 11529215046068469760,
  'Diamond': 0,
  'Gold': 109951162777600,
  'Silver': 0,
  'Bronze': 29686813949952},
 'usage': {'Super Saiyan': 959028051450,
  'Diamond': 0,
  'Gold': 11370725711,
  'Silver': 0,
  'Bronze': 92960700006},
 'percent_usage': {'Super Saiyan': 8.318242374853041e-06,
  'Diamond': 0,
  'Gold': 0.010341614789467712,
  'Silver': 0,
  'Bronze': 0.3131380152909616}}

In [16]:
tier_counts

{'Super Saiyan': 10, 'Diamond': 0, 'Gold': 1, 'Silver': 0, 'Bronze': 27}

In [17]:
tier_capacity

{'Super Saiyan': 11529215046068469760,
 'Diamond': 0,
 'Gold': 109951162777600,
 'Silver': 0,
 'Bronze': 29686813949952}

In [21]:
tier_percent_usage = {k: 100 * (v / tier_capacity[k]) for k,v in tier_usage.items() if tier_capacity[k] > 0}
tier_percent_usage

{'Super Saiyan': 8.318242374853041e-06,
 'Gold': 0.010341614789467712,
 'Bronze': 0.3131380152909616}

In [18]:
tier_usage

{'Super Saiyan': 959028051450,
 'Diamond': 0,
 'Gold': 11370725711,
 'Silver': 0,
 'Bronze': 92960700006}

In [12]:
total_requests = sum([int(v.get('total_successes', 0)) for k,v in (await get_miner_statistics(r)).items()])
total_requests

288879

In [8]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('network_data.db')
cursor = conn.cursor()

# Create network_stats table
cursor.execute('''
CREATE TABLE IF NOT EXISTS network_stats (
    total_network_capacity_mb INTEGER,
    current_validator_storage_mb INTEGER
)
''')

# Create validator_stats table
cursor.execute('''
CREATE TABLE IF NOT EXISTS validator_stats (
    validator_key TEXT PRIMARY KEY,
    store_attempts INTEGER,
    store_successes INTEGER,
    challenge_successes INTEGER,
    challenge_attempts INTEGER,
    retrieve_successes INTEGER,
    retrieve_attempts INTEGER,
    total_successes INTEGER,
    tier TEXT,
    storage_limit INTEGER
)
''')

conn.commit()


In [9]:
for key, values in (await get_miner_statistics(r)).items():
    cursor.execute('''
    INSERT INTO validator_stats (validator_key, store_attempts, store_successes, challenge_successes, challenge_attempts, retrieve_successes, retrieve_attempts, total_successes, tier, storage_limit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (key, 
         values['store_attempts'], 
         values['store_successes'], 
         values['challenge_successes'], 
         values['challenge_attempts'], 
         values['retrieve_successes'], 
         values['retrieve_attempts'], 
         values['total_successes'], 
         values['tier'], 
         values['storage_limit']))

conn.commit()

In [10]:
total_network_capacity = await get_network_capacity(r) / (1024 ** 2) # MB
current_storage = await current_validator_storage(r) / (1024 ** 2) # MB

cursor.execute('''
INSERT INTO network_stats (total_network_capacity, current_validator_storage)
VALUES (?, ?)
''', (total_network_capacity, current_storage))

conn.commit()

conn.close()

In [11]:
# Connect to the SQLite database
conn = sqlite3.connect('network_data.db')
cursor = conn.cursor()

# Retrieve and display data from network_stats
cursor.execute('SELECT * FROM network_stats')
network_stats = cursor.fetchall()
print("Network Stats:")
for row in network_stats:
    print(f"Total Network Capacity: {row[0]}, Current Validator Storage: {row[1]}")

# Retrieve and display data from validator_stats
cursor.execute('SELECT * FROM validator_stats')
validator_stats = cursor.fetchall()
print("\nValidator Stats:")
for row in validator_stats:
    print(f"Validator Key: {row[0]}, Store Attempts: {row[1]}, Store Successes: {row[2]}, Challenge Successes: {row[3]}, Challenge Attempts: {row[4]}, Retrieve Successes: {row[5]}, Retrieve Attempts: {row[6]}, Total Successes: {row[7]}, Tier: {row[8]}, Storage Limit: {row[9]}")

# Close the database connection
conn.close()


Network Stats:
Total Network Capacity: 10995249446912, Current Validator Storage: 1014098.6224813461
Total Network Capacity: 10995249446912, Current Validator Storage: 1014098.6224813461

Validator Stats:
Validator Key: 5Cih6V8oc7ahYHAXwnSVu8ys8h3GM4A1kXJqfJFQCwD9XmFy, Store Attempts: 128, Store Successes: 128, Challenge Successes: 73, Challenge Attempts: 78, Retrieve Successes: 7, Retrieve Attempts: 10, Total Successes: 2261, Tier: Gold, Storage Limit: 109951162777600
Validator Key: 5CoEiuGiXZgeFSpccqo17aiovSRh9Tv2Vn6j65J93iEhB2hJ, Store Attempts: 0, Store Successes: 0, Challenge Successes: 0, Challenge Attempts: 81, Retrieve Successes: 0, Retrieve Attempts: 0, Total Successes: 1998, Tier: Bronze, Storage Limit: 1099511627776
Validator Key: 5Do7YsNrKEpejfTMsmaCxh9FPHrxgBi4jYh9VcbeifobjNAe, Store Attempts: 0, Store Successes: 0, Challenge Successes: 0, Challenge Attempts: 0, Retrieve Successes: 0, Retrieve Attempts: 0, Total Successes: 0, Tier: Bronze, Storage Limit: 1099511627776
Vali

In [ ]:
"""
Steps:
- Connect to SQL database on the server
- Create tables for network stats and validator stats
- Insert data into the tables
- Retrieve data from the tables and display it


Stat ideas:
- Current tier usage vs total capacity (per tier per validator)
- Current miner usgae vs total capacity (per miner UID/Hotkey across all miners)
- Current validator usage vs total capcity (per validator UID/hotkey across all miners)
- % usage of each tier
- % usage of each miner
- % usage of each validator
- Tier counts (how many miners in each tier)
- Tier capacity (how much storage in each tier)
- Tier usage (how much storage is being used in each tier)  
- Total theoretical network capacity (overall across all validators, they have their own copy therefore will have independent caps for each hotkey)
- Current total challenges successes vs total challenge attempts
- Current total store successes vs total store attempts
- Current total retrieve successes vs total retrieve attempts
"""